In [1]:
import neo
import quantities as pq
import flylib
import db_access as dba
reload(dba)

<module 'db_access' from '/media/analysis-code/flight-muscles/src/db_access.pyc'>

In [2]:
#starfield1_group = [151,153,154,155,156,157,158,159,160,161,162,163,165,166,167,168,169]
sin_yaw_group = [225,226,227,228,229,230,231]

In [3]:
fly_db = dba.get_db()
swarm = flylib.Squadron(fly_db,sin_yaw_group)

In [4]:
keylist = ['b1', 'b2', 'b3', 'i1', 'i2', 'iii1', 'iii234','hg1', 'hg234' ]

In [5]:
import cPickle
flynum = 231
idx = squeeze(argwhere(array(sin_yaw_group) == 231))
fly = swarm.flies[idx]
expmnt = fly.experiments.values()[0]
f = open(expmnt.fly_path + expmnt.experiment_name + '_roi.cpkl')
roi_data = cPickle.load(f)
f.close()

In [6]:
xpos = np.array(expmnt.exp_record['tiff_data']['axon_framebase']['Xpos'])[sta:stp]*2*pi/10
plot(roi_data['b2'])

NameError: name 'sta' is not defined

In [7]:
sta,stp = (-8000,-1)
#sta,stp = (2000,5000)
xpos = np.array(expmnt.exp_record['tiff_data']['axon_framebase']['Xpos'])[sta:stp]*2*pi/10
lwing_angles = np.array(expmnt.exp_record['tiff_data']['axon_framebase']['Ph0'])[sta:stp]/5
rwing_angles = np.array(expmnt.exp_record['tiff_data']['axon_framebase']['Ph1'])[sta:stp]/5
lmr_angles = np.array(expmnt.exp_record['tiff_data']['axon_framebase']['Ph2'])[sta:stp]/5
#plot(xpos)
imgs = np.array(expmnt.exp_record['tiff_data']['images'][sta:stp,:,:])
from scipy import ndimage
imgs = ndimage.interpolation.rotate(imgs, 180, axes=(1, 2), reshape=True)
imgs = imgs/float(max(imgs.ravel()))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid.inset_locator import inset_axes
from matplotlib import animation

def convert_to_overlay(maximg,curr_img):
    from skimage import data, color, io, img_as_float
    gama = lambda x: x**1.1
    img_color = np.dstack((maximg, maximg, maximg))
    overlay = cm.jet(gama(curr_img))[:,:,:3]
    img_hsv = color.rgb2hsv(img_color)
    overlay_hsv = color.rgb2hsv(overlay)
    return cm.gray(curr_img)
    #alpha = 0.4
    #img_hsv[..., 0] = overlay_hsv[..., 0]
    #img_hsv[..., 1] = overlay_hsv[..., 2]*overlay_hsv[..., 1] * alpha
    #img_hsv[..., 2] = overlay_hsv[..., 2] * alpha
    #img_overlay = color.hsv2rgb(img_hsv)
    #return(img_overlay)

fly_diagram = dict()
aspect = shape(imgs[0][:,:50:-1])[0]/float(shape(imgs[0][:,:50:-1])[1])
fig = figure(figsize = (10,10*aspect))
img_ax = axes()

maximg = np.max(imgs,axis = 0)
gama = lambda x: x**1.0
maximg = gama(maximg)

img = imshow(convert_to_overlay(maximg,imgs[0])[:,:50:-1],aspect = 'auto')
img.set_interpolation('bicubic')
#img.set_clim(0,1)
inset_axes = inset_axes(img_ax,
                        width=3, # width = 30% of parent_bbox
                        height=3, # height : 1 inch
                        loc=1)
inset_axes.set_frame_on(False)
inset_axes.get_axes().set_axis_off()
img_ax.get_axes().set_axis_off()
    
wing_len = 9.0
theta_r = pi/4
theta_l = pi/4.3
stripe_center = 90
swidth = 3
arena_rad = 2*wing_len 

base = mpatches.Circle((0,0),arena_rad - swidth,facecolor = 'gray')
panels = mpatches.Circle((0,0),arena_rad,facecolor = 'y')
body = mpatches.Ellipse((0,0),2.5,3.5,facecolor = 'r')
abdomin = mpatches.Ellipse((0,-2.5),3,5,facecolor = 'r')
head = mpatches.Ellipse((0,2.0),3,2,facecolor = 'r')
stripe = mpatches.Wedge((0,0),arena_rad,stripe_center-5,stripe_center+5,width = swidth,color = 'k')
rwing = mpatches.Wedge((1,0),wing_len,-90,rad2deg(theta_r),alpha = 0.5,facecolor = 'r')
lwing = mpatches.Wedge((-1,0),wing_len,180-rad2deg(theta_l),-90,alpha = 0.5,facecolor = 'r')

fly_diagram.update({'base':base,'panels':panels,
                    'body':body,'abdomin':abdomin,
                    'head':head,'stripe':stripe,
                    'rwing':rwing,'lwing':lwing})

inset_axes.add_patch(panels)
inset_axes.add_patch(base)
inset_axes.add_patch(abdomin)
inset_axes.add_patch(head)
inset_axes.add_patch(body)
inset_axes.add_patch(rwing)
inset_axes.add_patch(lwing)
inset_axes.add_patch(stripe)

inset_axes.set_ybound(-20,20)
inset_axes.set_xbound(-20,20)
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

def init():
    return [rwing,lwing,stripe,img]

def animate(i):
    #print i
    theta_l,theta_r,stripe_center,frame = lwing_angles[i],rwing_angles[i],xpos[i],imgs[i]
    frame = convert_to_overlay(maximg[:,:50:-1],frame[:,:50:-1])
    stripe_center = rad2deg(stripe_center)+(180-rad2deg(stripe_center))*2
    rwing.set_theta2(rad2deg(theta_r))
    lwing.set_theta1(180-rad2deg(theta_l))
    stripe.set_theta1(stripe_center-5)
    stripe.set_theta2(stripe_center+5)
    img.set_data(frame)
    return [rwing,lwing,stripe,img]

frames = range(1,7000,2)
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=frames, interval=1, blit=True)
anim.save('fly%s.mov'%(flynum), fps=35,extra_args=['-vcodec', 'mpeg4', '-b', '5000k' ])
show()

In [283]:
plot(roi_data['b2'][sta:stp])

In [304]:
imshow(maximg[:,:50:-1])

In [63]:
imshow(curve(imgs[4]/max(imgs[4].ravel())),cmap = cm.gray)

In [255]:
shape(cm.jet(imgs[1]))

(130, 174, 3)

In [266]:
imshow(np.max(imgs,axis = 0))

In [325]:

imshow(gama(maximg),cmap = cm.gray)

NameError: name 'tpoint' is not defined

In [126]:
normimg = imgs[4]/float(max(imgs.ravel()))

In [92]:
lns = hist(curve(normimg).ravel(),bins = 100)

In [125]:
imshow(curve(normimg),cmap = cm.gray)

In [124]:

plot(curve(linspace(0,1,100)))

In [7]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid.inset_locator import inset_axes
from matplotlib import animation

fly_diagram = dict()
aspect = shape(imgs[0])[0]/float(shape(imgs[0])[1])
fig = figure(figsize = (10,10*aspect))
img_ax = axes()

img = imshow(imgs[0,:,:],cmap = cm.gray)
inset_axes = inset_axes(img_ax,
                        width=2, # width = 30% of parent_bbox
                        height=2, # height : 1 inch
                        loc=2)
inset_axes.set_frame_on(False)
inset_axes.get_axes().set_axis_off()
img_ax.get_axes().set_axis_off()
    
wing_len = 9.0
theta_r = pi/4
theta_l = pi/4.3
stripe_center = 90
swidth = 3
arena_rad = 2*wing_len 

base = mpatches.Circle((0,0),arena_rad - swidth,facecolor = 'gray')
panels = mpatches.Circle((0,0),arena_rad,facecolor = 'y')
body = mpatches.Ellipse((0,0),2.5,3.5,facecolor = 'r')
abdomin = mpatches.Ellipse((0,-2.5),3,5,facecolor = 'r')
head = mpatches.Ellipse((0,2.0),3,2,facecolor = 'r')
stripe = mpatches.Wedge((0,0),arena_rad,stripe_center-5,stripe_center+5,width = swidth,color = 'k')
rwing = mpatches.Wedge((1,0),wing_len,-90,rad2deg(theta_r),alpha = 0.5,facecolor = 'r')
lwing = mpatches.Wedge((-1,0),wing_len,180-rad2deg(theta_l),-90,alpha = 0.5,facecolor = 'r')

fly_diagram.update({'base':base,'panels':panels,
                    'body':body,'abdomin':abdomin,
                    'head':head,'stripe':stripe,
                    'rwing':rwing,'lwing':lwing})

inset_axes.add_patch(panels)
inset_axes.add_patch(base)
inset_axes.add_patch(abdomin)
inset_axes.add_patch(head)
inset_axes.add_patch(body)
inset_axes.add_patch(rwing)
inset_axes.add_patch(lwing)
inset_axes.add_patch(stripe)

inset_axes.set_ybound(-20,20)
inset_axes.set_xbound(-20,20)
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)


In [9]:
frames = range(1,5000,2)

In [10]:
frames

[1,
 3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 49,
 51,
 53,
 55,
 57,
 59,
 61,
 63,
 65,
 67,
 69,
 71,
 73,
 75,
 77,
 79,
 81,
 83,
 85,
 87,
 89,
 91,
 93,
 95,
 97,
 99,
 101,
 103,
 105,
 107,
 109,
 111,
 113,
 115,
 117,
 119,
 121,
 123,
 125,
 127,
 129,
 131,
 133,
 135,
 137,
 139,
 141,
 143,
 145,
 147,
 149,
 151,
 153,
 155,
 157,
 159,
 161,
 163,
 165,
 167,
 169,
 171,
 173,
 175,
 177,
 179,
 181,
 183,
 185,
 187,
 189,
 191,
 193,
 195,
 197,
 199,
 201,
 203,
 205,
 207,
 209,
 211,
 213,
 215,
 217,
 219,
 221,
 223,
 225,
 227,
 229,
 231,
 233,
 235,
 237,
 239,
 241,
 243,
 245,
 247,
 249,
 251,
 253,
 255,
 257,
 259,
 261,
 263,
 265,
 267,
 269,
 271,
 273,
 275,
 277,
 279,
 281,
 283,
 285,
 287,
 289,
 291,
 293,
 295,
 297,
 299,
 301,
 303,
 305,
 307,
 309,
 311,
 313,
 315,
 317,
 319,
 321,
 323,
 325,
 327,
 329,
 331,
 333,
 335,
 337,
 339,
 341,
 343,
 345,
 347,
 349,
 351,

In [47]:
plot(roi_data['b3'][-5000:])
plot(rwing_angles)

In [12]:
fly_db.close()

<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>


In [63]:
import matplotlib.gridspec as gridspec
gs = gridspec.GridSpec(6,6)
ax_img = subplot(gs[0:,0:])
imshow(imgs[0,:,:])
ax_fly = subplot(gs[5:,5:])
plot([1,3,4])

In [503]:
figure();plot(xpos)

In [342]:
rx = hstack([0,cos(linspace(theta_r,-pi/2,20))])+1
ry = hstack([0,sin(linspace(theta_r,-pi/2,20))])
plot(rx,ry)
lx = hstack([0,-1*cos(linspace(theta_l,-pi/2,20))])-1
ly = hstack([0,sin(linspace(theta_l,-pi/2,20))])
plot(lx,ly)

In [366]:
wd = mpatches.Wedge((0,0),5,pi/3,-1*pi/2)

In [422]:
inset_axes.set_visible(True)

In [424]:
inset_axis = inset_axes.get_axes()

In [426]:
inset_axis.set_axis_off()

In [427]:
draw()

In [399]:
plot(lmr_angles)

In [400]:
plot(lwing_angles - rwing_angles)

In [401]:
plot(xpos)